# Computes LIME explaination
Uses the AIX360 AI Exmplainability 360 toolkit to compute image classification explainations heathmaps

In [1]:
!pip3 install scikit-learn==0.24.1 aif360==0.3.0  tensorflow==2.4.0 nodejs==0.1.1 ipywidgets==7.6.3 lime==0.2.0.1 wget==3.2 #aix360==0.2.1 

In [2]:
import wget
wget.download('https://raw.githubusercontent.com/elyra-ai/component-library/master/claimed_utils.py')

'claimed_utils.py'

In [3]:
import os
import tensorflow as tf
from tensorflow import keras
from claimed_utils import unzip
import os.path
import glob
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np

In [ ]:
# @dependency codait_utils.ipynb
# @dependency model.zip
# @dependency data.zip
# @param model zip file name
# @param data zip file name
# @returns single random heatmap image object inline in jupyter
# (next version will return a zip folder with all images+heatmap)

In [4]:
model_zip = os.environ.get('model_zip', 'model.zip')
data_zip = os.environ.get('data_zip', 'data.zip')

In [ ]:
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [ ]:
unzip('.', model_zip)
unzip('.', data_zip)

In [ ]:
model = keras.models.load_model('model')

In [ ]:
folder = glob.glob("data/*")
num_classes = len(folder)

batch_size = 32
img_height = 400
img_width = 400
input_shape = (img_width, img_height)


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
val_ds_shuffeled = val_ds.shuffle(1)
for image, label in val_ds:
    break
label = label[0]
print(len(label))
label = tf.reshape(label, [1, len(label)])
print(label)
tf.math.argmax(label, 1)
label = tf.math.argmax(label, 1).numpy()[0]  # de-onehotencode

In [ ]:
def get_random_image():
    for image, label in val_ds:
        break
    image = image.numpy().astype(int)
    image = image[0]
    label = label[0]
    label = tf.reshape(label, [1, len(label)])
    image = image.reshape(img_height, img_width, 3)
    image_for_model = image.reshape(1, img_height, img_width, 3)
    label = tf.math.argmax(label, 1).numpy()[0]  # de-onehotencode
    prediction = model.predict(image_for_model)
    prediction = tf.math.argmax(prediction, 1).numpy()[0]  # de-onehotencode
    return image, label, prediction

In [ ]:
def get_explaination_as_mask(image, label):
    segmentation_fn = SegmentationAlgorithm(algo_type='slic')
    explanation = explainer.explain_instance(
        image,
        model.predict,
        segmentation_fn=segmentation_fn
    )
    print(label)
    return explanation.get_image_and_mask(label)[1]

In [ ]:
def merge_image_with_mask(image, mask):
    np_mask = np.array(mask.astype(int)*255, np.uint8)
    np_merged = np.empty(image.shape, np.uint8)
    np_merged[:, :, 0] = np.maximum(image[:, :, 0], np_mask)
    np_merged[:, :, 1] = np.maximum(image[:, :, 1], np_mask)
    np_merged[:, :, 2] = np.maximum(image[:, :, 2], np_mask)
    return np_merged

In [ ]:
fig = plt.figure(figsize=(12., 12.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(1, 2),
                 axes_pad=0.1,  # pad between axes in inch.
                 )

for ax in grid:
    image, label, prediction = get_random_image()
    print("label: "+str(label))
    mask = get_explaination_as_mask(image, label)
    masked_image = merge_image_with_mask(image, mask)
    ax.imshow(masked_image)

plt.show()